In [1]:
from fsr_trainable import FSR_Trainable
import fsr_data
import numpy as np
import warnings
import ray, logging
import ray.tune
np.seterr(divide='ignore', invalid='ignore')
display(ray.init(log_to_driver=False, logging_level=logging.ERROR, ignore_reinit_error=True))
warnings.filterwarnings("ignore", category=DeprecationWarning)

Python version:,3.10.12
Ray version:,2.5.1
Dashboard:,http://127.0.0.1:8265


In [2]:
from fsr_utility import *

In [3]:
data = fsr_data.get_data()

In [4]:
FSR_CENTER = np.array([
    (15*0.5, 14*0.5),
    (15*0.5, 24*0.5),
    (7*0.5, 24*0.5),
    (11*0.5, 51*0.5),
    (4.4, 7),
    (7.5, 18.5)
])
FSR_CENTER_COP = FSR_CENTER.mean(axis=0)
grfs = []
cops = []

for task in [1, 4]:
    for model in ['ANN', 'CNN-LSTM', 'LSTM']:
        print(f'search task: {task}, model: {model}')
        path = '/home/seokj/ray_results/' + TASKS(model, task)
        result_grid = ray.tune.Tuner.restore(path, FSR_Trainable).get_results()
        for result in result_grid:
            if result.best_checkpoints:
                checkpoint, metric = result.best_checkpoints[-1]
                trainer = FSR_Trainable(config=metric['config'])
                trainer.restore(checkpoint)
                preds, ys = trainer.eval()

                # data definition
                pred_force = preds[:, :6]
                pred_x_coord = preds[:, 6:12]
                pred_y_coord = preds[:, 12:]
                y_force = ys[:, :6]
                y_x_coord = ys[:, 6:12]
                y_y_coord = ys[:, 12:]

                # get grf and cop
                pred_grf = get_GRF(pred_force)
                y_grf = get_GRF(y_force)
                pred_x_cop, pred_y_cop = get_CoP(pred_force, pred_x_coord, pred_y_coord)
                y_x_cop, y_y_cop = get_CoP(y_force, y_x_coord, y_y_coord)

                # fill nan
                pred_x_cop[np.isnan(pred_x_cop) & ~np.isnan(y_x_cop)] = FSR_CENTER_COP[0]
                y_x_cop[~np.isnan(pred_x_cop) & np.isnan(y_x_cop)] = FSR_CENTER_COP[0]
                pred_y_cop[np.isnan(pred_y_cop) & ~np.isnan(y_y_cop)] = FSR_CENTER_COP[1]
                y_y_cop[~np.isnan(pred_y_cop) & np.isnan(y_y_cop)] = FSR_CENTER_COP[1]

                # get rmse of grf and cop
                grf = (((pred_grf - y_grf) ** 2).mean()) ** 0.5
                cop = ((pred_x_cop - y_x_cop) ** 2 + (pred_y_cop - y_y_cop) ** 2).mean(axis=0) ** 0.5
                grfs.append((grf, checkpoint.path, task, model))
                cops.append((cop, checkpoint.path, task, model))
                print(grf, cop)


search task: 1, model: ANN
363.5260049157419 5.387597159728776
845.5542191367742 6.4704569900176265
497.2490887372243 9.958455573147061
112.1023397182793 25.992543251623147
1028.586530147075 12.019004396155477
300.03839598041446 8.477536306390665
616.0330246829304 7.058084643718197
505.86395774951194 9.852705886884934
1072.777120374964 6.083324101410572
974.8960842059014 6.251492741659662
981.1563012588769 6.007143218978578
1029.514205827195 5.751781353547236
906.9352926201516 12.252961715470315
573.4874944146559 26.899603790428426
309.3210559030859 19.162419033688213
378.1035737731131 6.994216164333068
504.54266110508433 6.779189961523709
313.49566384561047 12.223355959318338
588.7911875189709 6.4599472818497015
982.4896309885413 4.373606868733898
307.42539745440683 10.560288417807781
306.08504486090135 9.55441920105724
711.0386768664557 4.248157101659022
569.5675442825021 10.564803555093434
378.0301533938794 5.918452177037084
834.9890343591345 12.502106756056907
312.4455702598454 8.5

In [5]:
from fsr_predictor import get_best_checkpoint

for task in [2, 5]:
    for model in ['ANN', 'CNN-LSTM', 'LSTM']:
        print(f'search task: {task}, model: {model}')
        try:
            checkpoint, metric = get_best_checkpoint(TASKS(model, task), 'rmse_force', 'min')
        except:
            checkpoint, metric = get_best_checkpoint(TASKS(model, task), 'rmse', 'min')
        trainer = FSR_Trainable(config=metric['config'])
        trainer.restore(checkpoint)
        preds, ys = trainer.eval()

        # data definition
        pred_force = preds
        y_force = ys

        # get grf and cop
        pred_grf = get_GRF(pred_force)
        y_grf = get_GRF(y_force)

        # get rmse of grf and cop
        grf = (((pred_grf - y_grf) ** 2).mean()) ** 0.5
        grfs.append((grf, checkpoint.path, task, model))
        print(grf)


search task: 2, model: ANN
455.96497946662527
search task: 2, model: CNN-LSTM
344.31510368411085
search task: 2, model: LSTM
381.03524689325
search task: 5, model: ANN
507.7086239911629
search task: 5, model: CNN-LSTM
249.96645087391227
search task: 5, model: LSTM
537.8578459965049


In [6]:
from fsr_predictor import get_best_checkpoint, get_best_prediction

experiment = min(((e[0], e[1]) for e in grfs if e[2] == 2), key=lambda e: e[1])[1].split('/')[-3]
try:
    output = get_best_checkpoint(experiment, 'rmse_force', 'min')
except:
    output = get_best_checkpoint(experiment, 'rmse', 'min')
BEST_GRF_TASK3_PRED, BEST_GRF_TASK3_Y = get_best_prediction(*output)

experiment = min(((e[0], e[1]) for e in grfs if e[2] == 5), key=lambda e: e[1])[1].split('/')[-3]
try:
    output = get_best_checkpoint(experiment, 'rmse_force', 'min')
except:
    output = get_best_checkpoint(experiment, 'rmse', 'min')
BEST_GRF_TASK6_PRED, BEST_GRF_TASK6_Y = get_best_prediction(*output)

In [7]:
for task in [3, 6]:
    for model in ['ANN', 'CNN-LSTM', 'LSTM']:
        print(f'search task: {task}, model: {model}')
        try:
            checkpoint, metric = get_best_checkpoint(TASKS(model, task), 'rmse_coord', 'min')
        except:
            checkpoint, metric = get_best_checkpoint(TASKS(model, task), 'rmse', 'min')
        trainer = FSR_Trainable(config=metric['config'])
        trainer.restore(checkpoint)
        preds, ys = trainer.eval()

        # data definition
        pred_force = BEST_GRF_TASK3_PRED if task == 3 else BEST_GRF_TASK6_PRED
        pred_x_coord = preds[:, :6]
        pred_y_coord = preds[:, 6:]
        y_force = BEST_GRF_TASK3_Y if task == 3 else BEST_GRF_TASK6_Y
        y_x_coord = ys[:, :6]
        y_y_coord = ys[:, 6:]

        # get grf and cop
        pred_grf = get_GRF(pred_force)
        y_grf = get_GRF(y_force)
        pred_x_cop, pred_y_cop = get_CoP(pred_force, pred_x_coord, pred_y_coord)
        y_x_cop, y_y_cop = get_CoP(y_force, y_x_coord, y_y_coord)

        # fill nan
        pred_x_cop[np.isnan(pred_x_cop) & ~np.isnan(y_x_cop)] = FSR_CENTER_COP[0]
        y_x_cop[~np.isnan(pred_x_cop) & np.isnan(y_x_cop)] = FSR_CENTER_COP[0]
        pred_y_cop[np.isnan(pred_y_cop) & ~np.isnan(y_y_cop)] = FSR_CENTER_COP[1]
        y_y_cop[~np.isnan(pred_y_cop) & np.isnan(y_y_cop)] = FSR_CENTER_COP[1]

        # get rmse of grf and cop
        grf = (((pred_grf - y_grf) ** 2).mean()) ** 0.5
        cop = ((pred_x_cop - y_x_cop) ** 2 + (pred_y_cop - y_y_cop) ** 2).mean(axis=0) ** 0.5
        grfs.append((grf, checkpoint.path, task, model))
        cops.append((cop, checkpoint.path, task, model))
        print(grf, cop)


search task: 3, model: ANN
455.96497946662527 16.225090771907585
search task: 3, model: CNN-LSTM
455.96497946662527 16.31701363859304
search task: 3, model: LSTM
455.96497946662527 16.18119541953184
search task: 6, model: ANN
507.7086239911629 4.281813535704353
search task: 6, model: CNN-LSTM
507.7086239911629 4.316217119725255
search task: 6, model: LSTM
507.7086239911629 4.354473024143468


In [8]:
best_grf_model = {}
best_cop_model = {}
for task in range(1, 7):
    for model in ['ANN', 'CNN-LSTM', 'LSTM']:
        best_grf_model[(task, model)] = min(((e[0], e[1]) for e in grfs if e[2] == task and e[3] == model), key=lambda e: e[1])
        if task not in [2, 5]:
            best_cop_model[(task, model)] = min(((e[0], e[1]) for e in cops if e[2] == task and e[3] == model), key=lambda e: e[1])
display(best_grf_model)
display(best_cop_model)

{(1, 'ANN'): (363.5260049157419,
  '/home/seokj/ray_results/FSR_Trainable_2023-10-12_18-45-12/FSR_Trainable_01479dbd_77_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-10-12_19-07-05/checkpoint_000100'),
 (1, 'CNN-LSTM'): (1173.7905477554332,
  '/home/seokj/ray_results/FSR_Trainable_2023-10-12_19-16-12/FSR_Trainable_07abda57_10_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-10-12_19-18-01/checkpoint_000002'),
 (1, 'LSTM'): (698.4928865063695,
  '/home/seokj/ray_results/FSR_Trainable_2023-10-12_20-17-01/FSR_Trainable_022d3b58_20_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-10-12_20-30-31/checkpoint_000004'),
 (2, 'ANN'): (455.96497946662527,
  '/home/seokj/ray_results/FSR_Trainable_2023-10-12_21-26-47/FSR_Trainable_80245c18_7_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,i

{(1, 'ANN'): (5.387597159728776,
  '/home/seokj/ray_results/FSR_Trainable_2023-10-12_18-45-12/FSR_Trainable_01479dbd_77_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-10-12_19-07-05/checkpoint_000100'),
 (1, 'CNN-LSTM'): (12.194643089010821,
  '/home/seokj/ray_results/FSR_Trainable_2023-10-12_19-16-12/FSR_Trainable_07abda57_10_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-10-12_19-18-01/checkpoint_000002'),
 (1, 'LSTM'): (5.69853732432444,
  '/home/seokj/ray_results/FSR_Trainable_2023-10-12_20-17-01/FSR_Trainable_022d3b58_20_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,imputer=sklearn_impute_SimpleI_2023-10-12_20-30-31/checkpoint_000004'),
 (3, 'ANN'): (16.225090771907585,
  '/home/seokj/ray_results/FSR_Trainable_2023-10-13_04-50-38/FSR_Trainable_6b29fcd7_100_criterion=torch_nn_MSELoss,data_loader=fsr_data_get_index_splited_by_time,

In [9]:
import pandas as pd
grf_frame = {}
cop_frame = {}
for i in range(1, 7):
    grf_frame[i] = [best_grf_model[(i, 'ANN')][0], best_grf_model[(i, 'CNN-LSTM')][0], best_grf_model[(i, 'LSTM')][0]]
for i in [1, 3, 4, 6]:
    cop_frame[i] = [best_cop_model[(i, 'ANN')][0], best_cop_model[(i, 'CNN-LSTM')][0], best_cop_model[(i, 'LSTM')][0]]
grf_frame = pd.DataFrame(grf_frame, index=['ANN', 'CNN-LSTM', 'LSTM'])
grf_frame.columns.set_names('Task', inplace=True)
grf_frame.index.set_names('Model', inplace=True)
display(grf_frame)
cop_frame = pd.DataFrame(cop_frame, index=['ANN', 'CNN-LSTM', 'LSTM'])
cop_frame.columns.set_names('Task', inplace=True)
cop_frame.index.set_names('Model', inplace=True)
display(cop_frame)

Task,1,2,3,4,5,6
Model,,,,,,
ANN,363.526005,455.964979,455.964979,700.942892,507.708624,507.708624
CNN-LSTM,1173.790548,344.315104,455.964979,1364.475998,249.966451,507.708624
LSTM,698.492887,381.035247,455.964979,798.903428,537.857846,507.708624


Task,1,3,4,6
Model,,,,
ANN,5.387597,16.225091,7.620214,4.281814
CNN-LSTM,12.194643,16.317014,8.595358,4.316217
LSTM,5.698537,16.181195,3.723472,4.354473
